In [1]:
# imports
import pandas as pd

# This tells matplotlib not to try opening a new window for each plot.
%matplotlib inline

# Import a bunch of libraries.
import json
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

# cnn imports
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
#from tensorflow.keras.layers.normalization import BatchNormalization
#from tensorflow.keras.layers.advanced_activations import LeakyReLU

from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import TimeDistributed
from keras.layers import Activation
from keras.layers import Layer
from keras.layers.embeddings import Embedding
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences


#from tensorflow.keras import backend as K
#from tensorflow.keras import layers
#from tensorflow.python.keras.layers import Lambda
#from tensorflow.keras.callbacks import TensorBoard
#from tensorflow.keras.backend import sparse_categorical_crossentropy
#from tensorflow.keras.layers import Dense, TimeDistributed

from sklearn.model_selection import train_test_split

from datetime import datetime

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Set the randomizer seed so results are the same each time.
np.random.seed(0)

Using TensorFlow backend.


In [2]:
# Load data
bodies_df = pd.read_csv("train_bodies.csv")
stances_df =  pd.read_csv("train_stances.csv")

# Examine data shape
bodies_df.shape
bodies_df.head()

stances_df.shape
stances_df.head()

stances_df.head()
#combine body and stance tables
combined = pd.concat([stances_df.set_index('Body ID'),bodies_df.set_index('Body ID')], axis=1, join='inner').reset_index()

combined.shape

combined.head()

combined_df_t=combined[['Headline', 'articleBody', 'Stance']]

combined_df_t.head()

# Extract column for just labels
labels_t = combined_df_t['Stance']


In [3]:
# Examine data shape
bodies_df.shape

(1683, 2)

In [4]:
stances_df.shape

(49972, 3)

In [5]:
combined_df_t.head()

,Headline,articleBody,Stance
0,Police find mass graves with at least '15 bodi...,Danny Boyle is directing the untitled film\n\n...,unrelated
1,Hundreds of Palestinians flee floods in Gaza a...,Hundreds of Palestinians were evacuated from t...,agree
2,"Christian Bale passes on role of Steve Jobs, a...",30-year-old Moscow resident was hospitalized w...,unrelated
3,HBO and Apple in Talks for $15/Month Apple TV ...,(Reuters) - A Canadian soldier was shot at the...,unrelated
4,Spider burrowed through tourist's stomach and ...,"Fear not arachnophobes, the story of Bunbury's...",disagree


In [7]:
#combined_df.to_csv("combined_stances.csv")

In [9]:
# Extract column for just labels
#labels = combined_df['Stance']

In [10]:
combined_df_t.shape

(49972, 3)

In [67]:
# Drop columns that are strings
all_data_processed = combined_df_t.drop(['Stance'], axis=1)
all_data_processed = combined_df_t.drop(['articleBody'], axis=1)

In [68]:
new_labels1= labels_t.copy()

new_labels1[new_labels1=='agree']=1
new_labels1[new_labels1=='unrelated']=0
new_labels1[new_labels1=='disagree']=0
new_labels1[new_labels1=='discuss']=0


In [69]:
new_labels1

0        0
1        1
2        0
3        0
4        0
        ..
49967    0
49968    0
49969    1
49970    0
49971    0
Name: Stance, Length: 49972, dtype: object

In [70]:
# Set some variables to hold test, dev, and training data.
test_data, test_labels = all_data_processed[1000:], new_labels1[1000:]
#dev_data, dev_labels = all_data_processed[17505:23340], labels[17505:23340]
train_data, train_labels = all_data_processed[:1000], new_labels1[:1000]

In [71]:
test_labels.head()

1000    0
1001    0
1002    0
1003    0
1004    0
Name: Stance, dtype: object

In [30]:
# Used GLoVE embeddings
embeddings_dict = {}
with open("glove.6B.50d.txt", 'r', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        token = values[0]
        vector = np.asarray(values[1:], "float32")
        embeddings_dict[token] = vector

In [31]:
def sent_to_ids(sentences):
    """
    converting a list of strings to a list of lists of word ids
    """
    
    # TODO - Update Regular Expression for parsing (TINA)
    # Improvement Option (Remove trailing 's') Check Missing list for optimization
    
    #re.sub('[^A-Za-z0-9]+', '', mystring)
    #cleanString = re.sub('\W+','', string ) 
    
    bad_chars = [';', ':', '!', "*"] 
    
    text_ids = []
    example = []
    sentences = sentences.replace('.','')
    sentences = sentences.replace('\'','')
    sentences = sentences.replace(',','')
    sentences = sentences.replace('$','')
    sentences = sentences.replace('/',' ')
    sentences = sentences.replace('','')
    for word in sentences.split(' '):
        #word.lower()
        temp = word.lower()
        #temp.replace('\'','')
        if temp in embeddings_dict:
            example.append(embeddings_dict[temp])
        else:
            print("Missing ", temp)            
    text_ids.append(example)

    return  text_ids   

In [32]:
all_data_emb = all_data_processed.copy()

In [34]:
#dont run
for i in range (1250):
    all_data_emb.at[i, 'Headline'] =  np.array(sent_to_ids(all_data_processed.at[i,'Headline']))
    all_data_emb.at[i, 'articleBody'] =  np.array(sent_to_ids(all_data_processed.at[i,'articleBody']))

Missing  film

seth
Missing  sony’s
Missing  biopic

danny
Missing  isaacsons
Missing  years

negotiations
Missing  it’s
Missing  talks

of
Missing  stage

insiders
Missing  list

wozniak
Missing  "woz"
Missing  storm

the
Missing  (wadi
Missing  gaza)
Missing  meters

gaza
Missing  down

the
Missing  110000
Missing  summer

the
Missing  entering

gaza
Missing  al-midana
Missing  sea

evacuated
Missing  al-bureij
Missing  strip

the
Missing  (wadi
Missing  gaza)
Missing  al-nuseirat
Missing  al-moghraqa
Missing  habesor
Missing  hebron

israeli
Missing  area

locals
Missing  hazard

this
Missing  dams

in
Missing  homes

in
Missing  lifenews
Missing  opoila
Missing  vessels



-
Missing  officials

at
Missing  testicles

-
Missing  adventures

-
Missing  say

interestingly
Missing  (reuters)
Missing  wednesday

the
Missing  montreal

(reporting
Missing  hopkins;
Missing  abocar)
Missing  arachnophobes
Missing  bunburys
Missing  "spiderman"
Missing  seemed

perth
Missing  online

earlie

Missing  brother

in
Missing  rungna
Missing  people’s
Missing  (ap
Missing  service)

the
Missing  nkis
Missing  jong’s
Missing  suggestion

she’s
Missing  report

“all
Missing  yo-jong
Missing  better”
Missing  decided

yo
Missing  yong-hui
Missing  “senior
Missing  official”
Missing  government

then
Missing  heung-gwang
Missing  nkis

“some
Missing  byong-so
Missing  people’s
Missing  yo-jong
Missing  korea”
Missing  said

the
Missing  coup

more:

north
Missing  lockdown?
Missing  attempt?
Missing  it:
Missing  cheese-obsessed
Missing  lardball
Missing  jong-un

the
Missing  dog-meat
Missing  bears

next
Missing  sheep’s
Missing  delicious

kim
Missing  it’s
Missing  money

the
Missing  podgy
Missing  salmond’s
Missing  year

while
Missing  it’s
Missing  they’ll
Missing  kim’s
Missing  ‘delicacies’
Missing  gruel

more:
Missing  dog-meat
Missing  farm

more:
Missing  sister

the
Missing  outlets

michael
Missing  leanings

‘the
Missing  elite’s
Missing  thoughts

‘despite
Missing 

Missing  9

cnn
Missing  head

a
Missing  recording

brown
Missing  shot

police
Missing  "made
Missing  connection"
Missing  familys
Missing  "assassinate
Missing  brown"
Missing  yo-jong
Missing  un’s
Missing  role?
Missing  ‘fabricated’
Missing  finally!
Missing  nfls
Missing  trademark

the
Missing  informative

understandably
Missing  espns
Missing  espns
Missing  pre-monday
Missing  is

[]
Missing  words:
Missing  "roger
Missing  liar"

you
Missing  make

you
Missing  programming

you
Missing  violence
what
Missing  month

[update:
Missing  tonights
Missing  "was
Missing  be"
Missing  "regular
Missing  kolber"]
Missing  iraq’s
Missing  iraq

“the
Missing  al-dulaimi
Missing  al-dulaimi
Missing  
Missing  explosions”
Missing  reuters

“the
Missing  al-dulaimi
Missing  esraa
Missing  mahel
Missing  al-qaisi
Missing  al-dulaimi”
Missing  said

maan
Missing  said

security
Missing  baghdadi’s
Missing  month

they
Missing  300000
Missing  bellagio’s
Missing  revealed

bloomberg
Missin

Missing  design
on
Missing  9to5mac
Missing  design

new
Missing  12in
Missing  9to5mac
advertisement

bear
Missing  (although
Missing  steeber)
Missing  9to5mac
Missing  "who
Missing  computer"

it
Missing  (to
Missing  instance)
Missing  category

sure
Missing  size:
Missing  12in
Missing  11in
Missing  shrunk

here
Missing  (according
Missing  rate)
Missing  12in
Missing  (left)
Missing  11in
Missing  (right):

new
Missing  12in
Missing  9to5mac

this
Missing  leaks:
Missing  (the
Missing  type-c
Missing  usb)
Missing  type-c
Missing  (or
Missing  dongle)
Missing  in

macworld
Missing  mcelhearn
Missing  magsafe
Missing  

as
Missing  keys

new
Missing  12in
Missing  9to5mac
advertisement

of
Missing  9to5macs
Missing  design

jack
Missing  legitimate

marsh
Missing  (more
Missing  below)
Missing  (its
Missing  84x26mm)

apparently
Missing  magsafe
Missing  reason:
Missing  magsafe
Missing  marshs
Missing  yet

the
Missing  him:
Missing  "the
Missing  lineup"

marshs
Missing  iphone

Missing  anniversary

sebastian
Missing  free-beacon
Missing  arabia

the
Missing  super-high
Missing  said

strike
Missing  again?
Missing  airplanes

the
Missing  added

us
Missing  down

since
Missing  syria

egypts
Missing  order

the
Missing  "jihadi
Missing  john"
Missing  emwazi
Missing  london

the
Missing  emwazi
Missing  programming
the
Missing  emwazi
Missing  question
police
Missing  reports
"we
Missing  investigation"
Missing  statement
in
Missing  isil
Missing  britons
the
Missing  emwazi
Missing  isil
Missing  butcher’s
Missing  cleaver

suresh
Missing  india’s
Missing  girl’s
Missing  screams

a
Missing  him

he
Missing  butcher’s
Missing  cleaver

it
Missing  street

kumar
Missing  attack

local
Missing  said:
Missing  ‘no
Missing  crime’

more:
Missing  robbery

‘we
Missing  it’s
Missing  stopped

‘this
Missing  punished’

a
Missing  said:
Missing  ‘people
Missing  hands

‘as
Missing  them

the
Missing  phelps’
Missing  god
who
Missing  figure
father
Missing  attack:


...

Missing  pro)
Missing  air’s
Missing  factor
profilel-r-copy
to
Missing  9to5mac
Missing  it’s
Missing  we’ve
Missing  mid-2015
now
Missing  read:
Missing  better:
Missing  air?
Missing  (reuters)
Missing  "islamist
Missing  strike-somali
Missing  government"
Missing  issued

the
Missing  story_number:
Missing  l5n0r62q3
Missing  story_date:
Missing  story_time:
Missing  (writing
Missing  blair;
Missing  hepinstall)
Missing  baghdad—an
Missing  group

the
Missing  al-baghdadi’s
Missing  leader’s
Missing  spouse

iraq’s
Missing  al-dulaimi
Missing  lebanon

mr
Missing  al-dulaimi
Missing  suspect

he
Missing  al-dulaimi

—copyright
Missing  2012

a
Missing  ago

a
Missing  “a
Missing  #america
Missing  (from
Missing  #islamicstate)”
Missing  states’
Missing  again

time
Missing  sotloff
Missing  2013

a
Missing  family’s
Missing  couldn’t
Missing  foley’s
Missing  fate

“we
Missing  answers”
Missing  “please
Missing  foleys
Missing  prayers”

a
Missing  “is
Missing  authenticity”

“if
M

Missing  report:
Missing  90%
Missing  sunlight

this
Missing  “we
Missing  purgin
Missing  n*gga
Missing  fam”

despite
Missing  “we
Missing  all”
Missing  news!

what
Missing  darkness?
Missing  ‘#6daysofdarkness‘
Missing  days!
Missing  isil
Missing  (cnn)
Missing  "newshour"
Missing  "a
Missing  troubles"

for
Missing  "war
Missing  beings"

and
Missing  reporting:
Missing  war

foley
Missing  again

a
Missing  recorded

for
Missing  foleys
Missing  theyd
Missing  disappearance

"we
Missing  people"
Missing  night

she
Missing  "an
Missing  person"

courageous
Missing  generous

foley
Missing  siblings

foley
Missing  syria

on
Missing  generosity

one
Missing  "funny
Missing  lebowski-loving
Missing  guy"

another
Missing  "dedication
Missing  understanding"

fisher
Missing  "jims
Missing  generosity"

he
Missing  2011

foley
Missing  jail

beheading
Missing  horrors

front-line
Missing  journalist

foley
Missing  "you
Missing  2011"

"i
Missing  be"
Missing  said

one
Missing  ha

Missing  267m
Missing  italy

while
Missing  22st
Missing  278m
Missing  delta

wels
Missing  62st
Missing  long

facebook
Missing  
Missing  
Missing  hooked:
Missing  stone

the
Missing  close

in
Missing  15st
Missing  spain

the
Missing  line

it
Missing  battle

he
Missing  lifenews
Missing  opoila
Missing  vessels



-
Missing  officials

at
Missing  testicles

-
Missing  adventures

-
Missing  say

interestingly
Missing  claimed

the
Missing  war

having
Missing  syria-based
Missing  warplanes

the
Missing  armies

fighter:
Missing  (pictured)

power:
Missing  full-equipped
Missing  raqqa)
Missing  forces

show
Missing  strength:
Missing  year

isis
Missing  air

they
Missing  rights

people
Missing  airport

the
Missing  border

it
Missing  models

leader:
Missing  (pictured)

declaration:
Missing  qaedas
Missing  year

islamic
Missing  kobane
Missing  trying

the
Missing  said

as
Missing  kobane
Missing  imminent

the
Missing  massacre

but
Missing  anti-isis
Missing  days

t

Missing  ‘killed’
Missing  negotiations

nigerian
Missing  arabia-based
Missing  danladi
Missing  ahmadu

the
Missing  “i
Missing  him”
Missing  bbc’s
Missing  
Missing  
Missing  “weve
Missing  feedback”
Missing  worldpost
Missing  email

ahmad
Missing  salkida
Missing  harams
Missing  group’s
Missing  “i
Missing  danladi
Missing  [sic]
Missing  he?"
Missing  salkida
Missing  twitter

but
Missing  omeri
Missing  worldpost
Missing  guy

“these
Missing  
Missing  wasn’t
Missing  [of
Missing  ahmadu’s
Missing  identity]
Missing  wouldn’t
Missing  nigeria”
Missing  omeri
Missing  "the
Missing  nigeria’s
Missing  confidence"

chad
Missing  chibok
Missing  nigerias
Missing  place

boko
Missing  harams
Missing  america’s
Missing  hausa-language
Missing  didn’t
Missing  themselves

on
Missing  ahamdu
Missing  precise

“on
Missing  chibok
Missing  oppression”
Missing  site’s
Missing  identity

as
Missing  omeri
Missing  schoolgirls

omeri
Missing  “they
Missing  said

asked
Missing  omeri
Miss

Missing  week

inaky
Missing  violence-plagued
Missing  "pits
Missing  remains"
Missing  city

"its
Missing  searched"
Missing  anonymity

the
Missing  site

the
Missing  chilpancingo

three
Missing  town

witnesses
Missing  away

the
Missing  graves

around
Missing  year

another
Missing  tip

"we
Missing  anxious"
Missing  vidulfo
Missing  missing

authorities
Missing  disappeared

the
Missing  "effective
Missing  diligent"
Missing  "one
Missing  times"

dozens
Missing  found

guerrero
Missing  "guerreros
Missing  unidos"
Missing  "los
Missing  rojos"
Missing  appended

a
Missing  i

putin
Missing  “this
Missing  world’s
Missing  danger”
Missing  “it
Missing  today”

correction:


In [53]:
all_data_emb

,Headline,articleBody
0,"[[[0.49725, -1.1949, 0.37137, -0.081662, 0.691...","[[[-0.31355, 0.42849, 0.35756, 0.42991, 0.2927..."
1,"[[[1.1569, -0.37447, 1.0013, -0.68139, -0.1584...","[[[1.1569, -0.37447, 1.0013, -0.68139, -0.1584..."
2,"[[[-0.014143, 0.22457, -0.76705, -1.3631, 1.35...","[[[-0.56148, -0.31749, 0.043532, -0.6, 0.54374..."
3,"[[[0.53654, 0.54846, -0.020284, 1.1767, 0.0991...","[[[-0.16768, 1.2151, 0.49515, 0.26836, -0.4585..."
4,"[[[1.5571, -0.36854, -0.37088, 0.79536, 0.6130...","[[[0.53661, -0.33872, 0.28772, -0.72584, -0.04..."
...,...,...
49967,Urgent: The Leader of ISIL 'Abu Bakr al-Baghda...,(CNN) -- Boko Haram laughed off Nigeria's anno...
49968,Brian Williams slams social media for speculat...,Along with unveiling the Apple Watch earlier t...
49969,Mexico Says Missing Students Not Found In Firs...,The bodies found in a mass grave were confirme...
49970,US Lawmaker: Ten ISIS Fighters Have Been Appre...,Rep. Duncan Hunter (R-CA) told Greta Van Suste...


In [54]:
all_data_processed['Headline'][1]

'Hundreds of Palestinians flee floods in Gaza as Israel opens dams'

In [76]:
test_data= test_data['Headline']

In [78]:
# prepare tokenizer
t = Tokenizer()
t.fit_on_texts(test_data)
vocab_size = len(t.word_index) + 1
# integer encode the documents
encoded_docs = t.texts_to_sequences(test_data)
#print(encoded_docs)

In [77]:
test_data

1000     6 hidden mass graves may hold missing Mexican ...
1001     Nigeria Says It Struck A Cease-Fire With Boko ...
1002     Audio of gunfire in Michael Brown case authent...
1003     Star Wars 7: JJ Abrams want to move release to...
1004     'The youngest martyr - the cub of al-Baghdadi'...
                               ...                        
49967    Urgent: The Leader of ISIL 'Abu Bakr al-Baghda...
49968    Brian Williams slams social media for speculat...
49969    Mexico Says Missing Students Not Found In Firs...
49970    US Lawmaker: Ten ISIS Fighters Have Been Appre...
49971    Shots Heard In Alleged Brown Shooting Recordin...
Name: Headline, Length: 48972, dtype: object

In [79]:
embeddings_index_ex = dict()
with open("glove.6B.50d.txt", 'r', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index_ex[word] = coefs
#    f.close()
print('Loaded %s word vectors.' % len(embeddings_index_ex))

Loaded 400000 word vectors.


In [80]:
t.word_index.items()

dict_items([('in', 1), ('to', 2), ('of', 3), ('isis', 4), ('the', 5), ('a', 6), ('is', 7), ('on', 8), ('for', 9), ('and', 10), ('by', 11), ('at', 12), ('after', 13), ('with', 14), ('foley', 15), ('james', 16), ('video', 17), ('journalist', 18), ('michael', 19), ('as', 20), ('says', 21), ('islamic', 22), ('state', 23), ('boko', 24), ('not', 25), ('was', 26), ('haram', 27), ('brown', 28), ('apple', 29), ('have', 30), ('claims', 31), ('audio', 32), ('american', 33), ('kim', 34), ('ebola', 35), ('his', 36), ('has', 37), ('us', 38), ('u', 39), ('s', 40), ('who', 41), ('jong', 42), ('man', 43), ('from', 44), ('watch', 45), ('report', 46), ('new', 47), ('it', 48), ('reportedly', 49), ('missing', 50), ('reports', 51), ('steve', 52), ('shooting', 53), ('militants', 54), ('kidnapped', 55), ('leader', 56), ('that', 57), ('jobs', 58), ('killed', 59), ('wright', 60), ('fighters', 61), ('dead', 62), ('are', 63), ('woman', 64), ('be', 65), ('hoax', 66), ('shot', 67), ('girls', 68), ('meteorite', 69),

In [81]:
# create a weight matrix for words in training docs
embedding_matrix_ex = np.zeros((vocab_size, 50))
for word, i in t.word_index.items():
    embedding_vector = embeddings_index_ex.get(word)
    if embedding_vector is not None:
        embedding_matrix_ex[i] = embedding_vector

In [82]:
vocab_size

3880

In [83]:
test_data

1000     6 hidden mass graves may hold missing Mexican ...
1001     Nigeria Says It Struck A Cease-Fire With Boko ...
1002     Audio of gunfire in Michael Brown case authent...
1003     Star Wars 7: JJ Abrams want to move release to...
1004     'The youngest martyr - the cub of al-Baghdadi'...
                               ...                        
49967    Urgent: The Leader of ISIL 'Abu Bakr al-Baghda...
49968    Brian Williams slams social media for speculat...
49969    Mexico Says Missing Students Not Found In Firs...
49970    US Lawmaker: Ten ISIS Fighters Have Been Appre...
49971    Shots Heard In Alleged Brown Shooting Recordin...
Name: Headline, Length: 48972, dtype: object

In [84]:
num_steps = 1
hidden_size = 10

model2 = Sequential()
#model2.add(Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32'))
#model.add(Embedding(vocab, hidden_size, input_length=num_steps))

#model2.add(Embedding(vocab_size, hidden_size, input_length=num_steps))
model2.add(Embedding(vocab_size, 50, weights=[embedding_matrix_ex], input_length=4, trainable=False))

#model2.add(LSTM(hidden_size, return_sequences=True))
#model2.add(LSTM(hidden_size, return_sequences=True))

#model.add(Conv2D(64, (3, 3)))

model2.add(Flatten())
#model2.add(Dense(1, activation='sigmoid'))
model2.add(Dense(1))
model2.add(Activation('sigmoid'))


#model2.add(Dropout(0.5))
#model.add(TimeDistributed(Dense(vocab_size)))
#model2.add(Activation('softmax'))

In [85]:
# summarize the model
print(model2.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 4, 50)             194000    
_________________________________________________________________
flatten_3 (Flatten)          (None, 200)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 201       
_________________________________________________________________
activation_3 (Activation)    (None, 1)                 0         
Total params: 194,201
Trainable params: 201
Non-trainable params: 194,000
_________________________________________________________________
None


In [86]:
# integer encode the documents
#encoded_docs = t.texts_to_sequences(all_data_processed['Headline'])
encoded_docs = t.texts_to_sequences(test_data)

print(encoded_docs)
# pad documents to a max length of 4 words
max_length = 4
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
print(padded_docs)

[[171, 2263, 132, 361, 84, 1218, 50, 183, 144], [90, 21, 48, 1198, 6, 309, 270, 14, 24, 27, 142, 63, 229, 1199, 2, 5, 543, 520], [32, 3, 422, 1, 19, 28, 394, 1118, 11, 364, 460], [273, 626, 505, 1700, 1015, 1629, 2, 2223, 105, 2, 917, 542], [1418, 881, 2371, 5, 1824, 3, 86, 746, 4, 2372, 6, 294, 206, 319, 461, 37, 71, 59, 20, 74, 595, 111, 791, 14, 36, 1140, 1, 99], [4, 61, 468, 85, 298, 462, 5, 39, 40, 183, 87], [869, 1468, 686, 6, 88, 217, 224, 119, 36, 146, 8, 375, 538], [466, 506, 134, 334, 112, 829, 11, 86, 1094, 13, 403, 260, 455, 1, 1361], [136, 323, 136, 324, 1, 15, 77, 199, 113, 1936, 783, 607, 122], [2288, 1537, 747, 1882, 514, 2289, 2111, 1, 1268, 747, 128, 312, 830], [37, 35, 701, 4, 54, 1, 329], [3210, 106, 687, 164, 25, 1440, 8, 302], [1991, 1992, 1993, 2, 249, 14, 35, 803, 13, 922, 14, 1701, 1702, 717, 411], [370, 396, 125, 68, 247, 197, 8, 368, 383, 125, 255, 333, 668], [4, 87, 985, 590, 413, 21, 313, 25, 910, 38, 120, 115], [627, 85, 894, 266, 8, 573], [6, 2939, 43, 29

In [87]:
model2.compile(optimizer='adam', loss='binary_crossentropy')

In [88]:
# fit the model
batch_size = 30
model2.fit(padded_docs, test_labels, batch_size=batch_size, epochs=50)




Epoch 1/50


InternalError: 2 root error(s) found.
  (0) Internal: Blas GEMV launch failed:  m=200, n=30
	 [[{{node dense_3/MatMul}}]]
  (1) Internal: Blas GEMV launch failed:  m=200, n=30
	 [[{{node dense_3/MatMul}}]]
	 [[loss_2/mul/_107]]
0 successful operations.
0 derived errors ignored.

In [751]:
results = model2.predict(padded_docs)

In [752]:
# evaluate the model
#loss, accuracy =
accuracy = model2.evaluate(padded_docs, new_labels, verbose=0)
print('Accuracy: %f' % (accuracy*100))

Accuracy: 65.200031


In [755]:
results[:20]

array([[0.38638723],
       [0.7668729 ],
       [0.44041687],
       [0.26257262],
       [0.34788358],
       [0.56804913],
       [0.23889074],
       [0.3207721 ],
       [0.390614  ],
       [0.44344983],
       [0.31266505],
       [0.38610175],
       [0.44149402],
       [0.6170564 ],
       [0.5334355 ],
       [0.38663054],
       [0.56980675],
       [0.41918647],
       [0.37718433],
       [0.4777925 ]], dtype=float32)

In [756]:
new_labels[:20]

0     0.25
1        1
2     0.25
3     0.25
4        0
5        1
6     0.25
7     0.25
8        1
9     0.25
10    0.75
11       1
12    0.25
13    0.25
14    0.75
15    0.25
16    0.75
17       1
18    0.25
19    0.75
Name: Stance, dtype: object

In [803]:
num_steps = 1
hidden_size = 35

model3 = Sequential()
#model2.add(Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32'))
#model.add(Embedding(vocab, hidden_size, input_length=num_steps))

#model2.add(Embedding(vocab_size, hidden_size, input_length=num_steps))
model3.add(Embedding(vocab_size, 50, weights=[embedding_matrix_ex], input_length=4, trainable=False))

model3.add(LSTM(hidden_size, return_sequences=True))
model3.add(LSTM(hidden_size, return_sequences=True))

#model.add(Conv2D(64, (3, 3)))

model3.add(Flatten())
#model2.add(Dense(1, activation='sigmoid'))
#model3.add(Dense(20))
model3.add(Dense(units=20, input_dim=35, activation='relu')) 
model3.add(Dropout(0.5))
#model3.add(Flatten())
model3.add(Dense(units=1, input_dim=20, activation='relu')) 

#model3.add(Activation('sigmoid'))


#model2.add(Dropout(0.5))
#model.add(TimeDistributed(Dense(vocab_size)))
#model2.add(Activation('softmax'))

In [804]:
# summarize the model
print(model3.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_47 (Embedding)     (None, 4, 50)             129000    
_________________________________________________________________
lstm_65 (LSTM)               (None, 4, 35)             12040     
_________________________________________________________________
lstm_66 (LSTM)               (None, 4, 35)             9940      
_________________________________________________________________
flatten_23 (Flatten)         (None, 140)               0         
_________________________________________________________________
dense_21 (Dense)             (None, 20)                2820      
_________________________________________________________________
dropout_29 (Dropout)         (None, 20)                0         
_________________________________________________________________
dense_22 (Dense)             (None, 1)                 21        
Total para

In [805]:
model3.compile(optimizer='adam', loss='binary_crossentropy')
#model3.compile(optimizer='adam', loss='sparse_categorical_crossentropy')

In [806]:
# fit the model
batch_size = 30
model3.fit(padded_docs, new_labels, batch_size=batch_size, epochs=50)

Epoch 1/50
1250/1250 [==============================] - 4s 3ms/step - loss: 0.8342
Epoch 2/50
1250/1250 [==============================] - 1s 584us/step - loss: 0.7309
Epoch 3/50
1250/1250 [==============================] - 1s 564us/step - loss: 0.7063
Epoch 4/50
1250/1250 [==============================] - 1s 578us/step - loss: 0.7103
Epoch 5/50
1250/1250 [==============================] - 1s 576us/step - loss: 0.7114
Epoch 6/50
1250/1250 [==============================] - 1s 562us/step - loss: 0.6937
Epoch 7/50
1250/1250 [==============================] - 1s 574us/step - loss: 0.6870
Epoch 8/50
1250/1250 [==============================] - 1s 577us/step - loss: 0.6940
Epoch 9/50
1250/1250 [==============================] - 1s 564us/step - loss: 0.6844
Epoch 10/50
1250/1250 [==============================] - 1s 575us/step - loss: 0.6910
Epoch 11/50
1250/1250 [==============================] - 1s 560us/step - loss: 0.6880
Epoch 12/50
1250/1250 [==============================] - 1s 586us

In [807]:
# evaluate the model
#loss, accuracy =
accuracy3 = model3.evaluate(padded_docs, new_labels, verbose=0)
print('Accuracy: %f' % (accuracy3*100))

Accuracy: 60.398601
